In [47]:
import matplotlib.pyplot as plt
import numpy as np
import time
import csv
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

In [10]:
path_to_dataset='/Users/wuxiaodong/Desktop/18fall/SpecialProblem/Rice/1st Floor Avg Space Humidity.csv'
sequence_length=20
max_values = 1343

In [11]:
    with open(path_to_dataset) as f:
        data = csv.reader(f)
        bike = []
        nb_of_values = 0
        for line in data:
            try:
                bike.append(float(line[1]))
                nb_of_values +=1
            except ValueError:
                pass
            if nb_of_values >= max_values:
                break

In [14]:
result = []
for index in range(len(bike) - sequence_length):
    result.append(bike[index: index + sequence_length])
result = np.array(result)

In [19]:
# normalize data
result_mean = result.mean()
result -= result_mean        
print("Shift : ", result_mean)
print("Data : ",result.shape)


('Shift : ', 43.637386945405886)
('Data : ', (1323, 20))


In [28]:
row = int(round(0.9 * result.shape[0]))
train = result[:row, :]
train

array([[ 1.15955672,  1.15955672,  0.38611922, ..., -1.75607362,
        -0.15607133, -0.15607133],
       [-0.94044557, -0.40763536, -0.40763536, ..., -0.63888231,
        -0.63888231,  0.41736922],
       [-0.04356828, -0.04356828,  0.03299269, ..., -0.00606599,
        -0.79513612, -0.79513612],
       ...,
       [-0.6060721 , -0.01231828, -0.01231828, ..., -0.62951341,
        -0.62951341,  0.18455062],
       [-2.24982515, -2.17638841, -2.17638841, ...,  0.18611464,
         0.18611464, -0.15294328],
       [ 2.61893782,  2.61893782, -0.17482057, ...,  2.4923738 ,
        -0.15763536, -0.15763536]])

In [29]:
np.random.shuffle(train)
train


array([[-0.29669633, -2.56232515, -2.56232515, ..., -2.61857667,
        -2.61857667, -0.02794328],
       [ 1.35018477,  1.35018477,  0.42986617, ...,  1.21893248,
         2.11268553,  2.11268553],
       [ 0.24861846, -1.37638536, -1.37638536, ...,  0.47518096,
         0.47518096,  0.39393172],
       ...,
       [-0.59825578, -0.59825578, -0.94825425, ...,  0.08299575,
         0.58455977,  0.58455977],
       [-0.07013383, -0.07013383, -0.07169786, ..., -0.15919557,
        -0.19044557, -0.19044557],
       [-1.31388536, -1.30450883, -1.30450883, ...,  2.23768553,
         2.23768553, -1.18263688]])

In [30]:
X_train = train[:, :-1]
y_train = train[:, -1]
X_test = result[row:, :-1]
y_test = result[row:, -1]

In [31]:
X_train

array([[-0.29669633, -2.56232515, -2.56232515, ...,  2.78612456,
        -2.61857667, -2.61857667],
       [ 1.35018477,  1.35018477,  0.42986617, ...,  1.21893248,
         1.21893248,  2.11268553],
       [ 0.24861846, -1.37638536, -1.37638536, ...,  1.73143706,
         0.47518096,  0.47518096],
       ...,
       [-0.59825578, -0.59825578, -0.94825425, ...,  0.08299575,
         0.08299575,  0.58455977],
       [-0.07013383, -0.07013383, -0.07169786, ..., -0.15919557,
        -0.15919557, -0.19044557],
       [-1.31388536, -1.30450883, -1.30450883, ...,  2.21580825,
         2.23768553,  2.23768553]])

In [34]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [56]:
X_test

array([[[ 1.37518248],
        [ 1.37518248],
        [-0.30763307],
        ...,
        [-1.3248221 ],
        [-1.3248221 ],
        [-0.30450883]],

       [[ 1.37518248],
        [-0.30763307],
        [-0.30763307],
        ...,
        [-1.3248221 ],
        [-0.30450883],
        [-0.30450883]],

       [[-0.30763307],
        [-0.30763307],
        [-0.34669557],
        ...,
        [-0.30450883],
        [-0.30450883],
        [ 0.75799117]],

       ...,

       [[ 0.36112151],
        [ 0.56893096],
        [ 0.56893096],
        ...,
        [ 0.77205596],
        [ 0.81267867],
        [ 0.81267867]],

       [[ 0.56893096],
        [ 0.56893096],
        [ 0.71424803],
        ...,
        [ 0.81267867],
        [ 0.81267867],
        [ 0.75487075]],

       [[ 0.56893096],
        [ 0.71424803],
        [ 0.71424803],
        ...,
        [ 0.81267867],
        [ 0.75487075],
        [ 0.75487075]]])

In [67]:
# build model
model = Sequential()
layers = [1, 50, 100, 1]
# layer1
model.add(LSTM(layers[1], # number of outputs
               input_shape=(None, 1),  # dimension of input
               return_sequences=True))
model.add(Dropout(0.2)) # 20% dropout


In [68]:
# layer 2
model.add(LSTM(layers[2], # number of outputs
               return_sequences=False))
model.add(Dropout(0.2)) # 20% dropout

In [69]:
# layer 3
model.add(Dense(layers[3])) # times a dense matrix
model.add(Activation("linear")) #linear regression

In [70]:
start = time.time()
model.compile(loss="mse", optimizer="rmsprop")
print("Compilation Time: ", time.time() - start)

('Compilation Time: ', 0.04622006416320801)


In [72]:
# run model
data = None
epochs = 30

model.fit(X_train, y_train, batch_size=512, nb_epoch=epochs, validation_split=0.05)
predicted = model.predict(X_test)

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """


Train on 1131 samples, validate on 60 samples
Epoch 1/30
1131/1131 [==============================] - 2s 2ms/step - loss: 1.4907 - val_loss: 1.2046
Epoch 2/30
1131/1131 [==============================] - 0s 364us/step - loss: 1.4669 - val_loss: 1.1988
Epoch 3/30
1131/1131 [==============================] - 1s 492us/step - loss: 1.4288 - val_loss: 1.2073
Epoch 4/30
1131/1131 [==============================] - 0s 379us/step - loss: 1.4074 - val_loss: 1.1988
Epoch 5/30
1131/1131 [==============================] - 1s 491us/step - loss: 1.3775 - val_loss: 1.1989
Epoch 6/30
1131/1131 [==============================] - 0s 419us/step - loss: 1.3462 - val_loss: 1.1676
Epoch 7/30
1131/1131 [==============================] - 0s 381us/step - loss: 1.3354 - val_loss: 1.1502
Epoch 8/30
1131/1131 [==============================] - 0s 425us/step - loss: 1.3119 - val_loss: 1.1236
Epoch 9/30
1131/1131 [==============================] - 0s 433us/step - loss: 1.2941 - val_loss: 1.0999
Epoch 10/30
1131/113

In [74]:
len(predicted)

132

In [88]:
y_test

array([-0.30450883,  0.75799117,  0.75799117, -1.29826036, -1.29826036,
       -1.29669633, -1.29669633, -0.22794786, -0.22794786, -0.26701036,
       -0.26701036,  0.4798654 ,  0.4798654 ,  0.65486464,  0.65486464,
       -1.2545096 , -1.2545096 , -0.0904471 , -0.0904471 ,  0.38299117,
        0.38299117, -0.10138383, -0.10138383, -1.26544633, -1.26544633,
        0.21267638,  0.21267638, -1.12794938, -1.12794938, -1.24982133,
       -1.24982133, -0.02013078, -0.02013078, -0.06856981, -0.06856981,
        0.0111154 ,  0.0111154 ,  0.17205367,  0.17205367, -1.1295096 ,
       -1.1295096 ,  0.05174193,  0.05174193, -0.14201036, -0.14201036,
       -1.14669862, -1.14669862,  0.04705748,  0.04705748,  0.31424269,
        0.31424269,  0.24080596,  0.24080596, -1.07638612, -1.07638612,
       -1.07638231, -1.07638231, -0.93888536, -0.93888536,  0.36424193,
        0.36424193, -0.90138307, -0.90138307,  0.37049422,  0.37049422,
       -0.90763536, -0.90763536, -0.95919862, -0.95919862,  0.45

In [78]:
y_test
np.shape(y_test)

(132,)

In [79]:
predicted = np.reshape(predicted, (predicted.size,))

In [93]:
plt.plot(predicted)
plt.show()

In [95]:
np.mean(y_test)


-0.04964249266928856

In [96]:
np.mean(predicted)

-0.16728672

In [100]:
print('R-Squared: %f'%(r2_score(y_test, predicted)))

R-Squared: 0.227968


In [99]:
from sklearn.metrics import r2_score
